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

In [16]:
import os
from crewai import LLM
from typing import Type
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, FirecrawlCrawlWebsiteTool, FirecrawlScrapeWebsiteTool

c:\Users\SAMAR\anaconda3\envs\agent\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\SAMAR\anaconda3\envs\agent\Lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
c:\Users\SAMAR\anaconda3\envs\agent\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Use

In [ ]:
load_dotenv()
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["FIRECRAWL_API_KEY"] = os.getenv("FIRECRAWL_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["EXA_API_KEY"] = os.getenv("EXA_API_KEY")
os.environ["OPENROUTER_API_KEY"] = os.getenv("OPENROUTER_API_KEY")

In [ ]:
# Instantiate the search tool
search_tool = SerperDevTool()

# Run a sample search query
results = search_tool.run(search_query="Kerala Tourusim")
print(results)

In [ ]:
scrape_tool = FirecrawlScrapeWebsiteTool(api_key="fc-599cd87f720148d29800f05c3be01424")

scrape_result = scrape_tool.run(url="https://www.keralatourism.org/")
print(scrape_result)

In [1]:
evaluation_data = {
  "clarity": {
    "score": 8.5,
    "explanation": "The message is clear and easy to understand, but could benefit from more concise wording."
  },
  "tone": {
    "score": 7.0,
    "explanation": "The tone aligns with the brand's voice but lacks the emotional or persuasive depth needed to fully resonate with the audience."
  },
  "engagement": {
    "score": 6.5,
    "explanation": "The post includes a question to prompt interaction, but the call to action is weak and could be more compelling."
  },
  "platform_alignment": {
    "score": 9.0,
    "explanation": "The post is well-suited for Instagram, with visual appeal and a conversational style that fits the platform's culture."
  },
  "hashtags": {
    "score": 5.5,
    "explanation": "Some hashtags are relevant, but others are overly generic or lack strategic alignment with the post's goals."
  },
  "overall_effectiveness": {
    "score": 7.3,
    "explanation": "Overall, the post is effective but has room for improvement. To improve, consider refining the tone to be more engaging, strengthening the call to action, and optimizing hashtag selection for better reach and relevance."
  }
}

In [6]:
import json

In [15]:
evaluation_data['overall_effectiveness']['explanation']

'Overall, the post is effective but has room for improvement. To improve, consider refining the tone to be more engaging, strengthening the call to action, and optimizing hashtag selection for better reach and relevance.'

In [2]:
from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://www.keralatourism.org/")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
url_list = ["https://www.keralatourism.org/", 
            "http://www.keralatourism.gov.in/", 
            "https://www.makemytrip.com/holidays-india/kerala-tourism.html"]

In [9]:
type(url_list)

list

In [14]:
def get_content(urls):
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return [doc.page_content for doc in docs]


get_content(url_list)

['\n\n\nExample Domain\n\n\n\n\n\n\n\nExample Domain\nThis domain is for use in illustrative examples in documents. You may use this\n    domain in literature without prior coordination or asking for permission.\nMore information...\n\n\n\n',
 'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchGoogle offered in:  हिन्दी বাংলা తెలుగు मराठी தமிழ் ગુજરાતી ಕನ್ನಡ മലയാളം ਪੰਜਾਬੀ AdvertisingBusiness SolutionsAbout GoogleGoogle.co.in© 2025 - Privacy - Terms   ']

In [ ]:
from typing import Type, List
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.document_loaders import WebBaseLoader

class GetContentInput(BaseModel):
    """Input schema for GetContentTool."""
    urls: List[str] = Field(..., description="List of URLs to fetch content from.")

class GetContentTool(BaseTool):
    name: str = "get_content"
    description: str = "Fetches and returns the content from the provided list of URLs."
    args_schema: Type[BaseModel] = GetContentInput

    def _run(self, urls: List[str]) -> List[str]:
        """Fetch content from the provided URLs."""
        loader = WebBaseLoader(urls)
        docs = loader.load()
        return [doc.page_content for doc in docs]

c:\Users\SAMAR\anaconda3\envs\agent\Lib\typing.py:1570: RuntimeWarning: coroutine 'WebBaseLoader.fetch_all' was never awaited
  return ((p, type(p)) for p in parameters)


In [ ]:
content = loader.load()[0].page_content

In [ ]:
content

In [ ]:
loader_multiple_pages = WebBaseLoader(
    ["https://www.example.com/", "https://google.com"]
)

In [ ]:
loader_multiple_pages.load()[1].page_content

In [ ]:
docs = []
for doc in loader.lazy_load():
    docs.append(doc)
print(docs[0].page_content)
print(docs[0].metadata)

In [ ]:
def web_base_loader(url: str) -> str:
    loader = WebBaseLoader(url)
    content = loader.load()[0].page_content
    return content

In [ ]:
web_base_loader("https://www.keralatourism.org/")

In [ ]:
crawl_tool = FirecrawlCrawlWebsiteTool(api_key="fc-599cd87f720148d29800f05c3be01424")

In [ ]:
crawl_tool._run(url="https://www.keralatourism.org/")

In [ ]:
tool.run(url='firecrawl.dev')

In [ ]:
from firecrawl import FirecrawlApp

class CustomFirecrawlTool:
    api_key="fc-1fa1181db69b4d2ea0c586b88ac34321"
    url="https://www.keralatourism.org/"
    def __init__(self, api_key):
        self.firecrawl_app = FirecrawlApp(api_key=api_key)

    def run(self, url):
        # Define your parameters here
        params = {
            # Add necessary parameters based on the latest API documentation
        }
        return self.firecrawl_app.crawl_url(url, params)


In [ ]:
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key='fc-1fa1181db69b4d2ea0c586b88ac34321')

# Crawl a website:
crawl_status = app.crawl_url(
  'https://firecrawl.dev', 
  params={
    'limit': 100, 
    'scrapeOptions': {'formats': ['markdown', 'html']}
  },
  poll_interval=30
)
print(crawl_status)

In [ ]:
from crewai_tools import FirecrawlCrawlWebsiteTool

tool = FirecrawlCrawlWebsiteTool(url='firecrawl.dev')

In [ ]:
# Install with pip install firecrawl-py
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key='fc-1fa1181db69b4d2ea0c586b88ac34321')

crawl_result = app.crawl_url('https://docs.mendable.ai', params={
'limit': 3,
'scrapeOptions': {
	'formats': [ 'markdown' ],
  }
})

In [ ]:
crawl_tool = FirecrawlCrawlWebsiteTool(api_key="fc-1fa1181db69b4d2ea0c586b88ac34321", max_depth=1)

crawl_result = crawl_tool.run(url="https://www.keralatourism.org/")
options = {
    "crawlerOptions": crawler_options,
    "timeout": timeout,
}
return self._firecrawl.crawl_url(url, options)

print(crawl_result)

In [ ]:
from firecrawl import FirecrawlApp

fc = FirecrawlApp(api_key="fc-1fa1181db69b4d2ea0c586b88ac34321")
res = fc.crawl_url(url="https://www.keralatourism.org/")
print(res)


In [ ]:
crawl_tool = FirecrawlCrawlWebsiteTool()
crawl_result = crawl_tool.run(url="https://www.keralatourism.org/")
print(crawl_result)

In [ ]:
# Install with pip install firecrawl-py
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key="fc-1fa1181db69b4d2ea0c586b88ac34321")

crawl_result = app.crawl_url('https://www.keralatourism.org/', params={
'limit': 5,
'scrapeOptions': {
	'formats': [ 'markdown' ],
  }
})

In [ ]:
crawl_result = crawl_tool.run(
    url="https://www.keralatourism.org/") 

In [ ]:
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from firecrawl import FirecrawlApp

class FirecrawlToolInput(BaseModel):
    """Input schema for FirecrawlTool."""
    website_url: str = Field(
        ..., 
        description="The URL of the website to scrape content from."
    )

class FirecrawlTool(BaseTool):
    name: str = "Firecrawl Web Scraper"
    description: str = (
        "A tool that scrapes content from websites and returns it in a format "
        "suitable for LLM processing. Useful for gathering information from "
        "specific web pages for research and content creation."
    )
    args_schema: Type[BaseModel] = FirecrawlToolInput

    def _run(self, website_url: str) -> str:
        try:
            firecrawl_instance = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
            result = firecrawl_instance.scrape_url(website_url)
            return result['markdown']
        except Exception as e:
            return f"Error scraping website: {str(e)}"

In [ ]:
serper_tool = SerperDevTool()

firecrawl_tool = FirecrawlTool()

# llm_creator = LLM(model="gemini/gemini-1.5-flash",
#                    verbose=True, 
#                    temperature=0.5,
#                    api_key=os.getenv("GEMINI_API_KEY"))

# llm_manager = LLM(model="gemini/gemini-1.5-flash",
#                    verbose=True, 
#                    temperature=0.5,
#                    api_key=os.getenv("GEMINI_API_KEY"))

llm_creator = LLM(model="ollama/llama3.2:3b",
                   verbose=True, 
                   temperature=0.5,
                   base_url="http://localhost:11434")

llm_manager = LLM(model="ollama/deepseek-r1:8b",
                  verbose=True,
                  temperature=0.3,
                  base_url="http://localhost:11434")

In [ ]:
from crewai_tools import FirecrawlCrawlWebsiteTool

tool = FirecrawlCrawlWebsiteTool(
    max_depth=2, 
    api_key="fc-e5b59793e6e8489eaa87d93c03aa98e4")

In [ ]:
import os
os.environ["FIRECRAWL_API_KEY"] = "fc-e5b59793e6e8489eaa87d93c03aa98e4"

In [ ]:
from firecrawl.firecrawl import FirecrawlApp

class CustomFirecrawlTool:
    def __init__(self, api_key):
        self.firecrawl = FirecrawlApp(api_key=api_key)

    def crawl(self, url):
        response = self.firecrawl.crawl_url(url)
        return response

In [ ]:
crawl = tool.run('https://www.keralatourism.org/')

In [ ]:
crawl = firecrawl_tool.run("https://www.keralatourism.org/")

In [ ]:
print(crawl)

In [ ]:
serper_search_agent = Agent(
    role="Google Search",
    goal="Perform a Google search for '{topic}' and extract all URL links from the JSON response.",
    backstory="""
    You are a search expert that retrieves structured search results. Your main responsibility is to extract URLs from the JSON response and store them into list.
    """,
    memory=True,
    llm=llm_creator,
    tools=[serper_tool],
    allow_delegation=True,
)

serper_search_task = Task(
    description="""
    Perform a Google search for '{topic}' using the Serper tool. Extract all URL links from the JSON response "
    and store them in a list.
    """,
    expected_output="A list of Top 5 URLs extracted from the JSON response.",
    agent=serper_search_agent,
    tools=[serper_tool],
    verbose=True,
)

research_agent = Agent(
    role="Content Researcher",
    goal="Process each URL to extract and format content, and select the top 2 to 4 image URLs (If Available).",
    backstory="You are skilled in extracting, summarizing, and organizing content and images from web pages.",
    verbose=True,
    memory=True,
    llm=llm_creator,
    allow_delegation=False,
)

research_task = Task(
    description="""
    Process each URL in the list using the `firecrawl_tool` to extract content. 
    For each URL, select the top 2 to 4 image URLs if available. 
    Merge all extracted content and provide a concise summary.
    """,
    expected_output="""
    A JSON object containing:
     - 'platform': The platform for which the content is being prepared.
     - 'content': Formatted and summarized text content.
     - 'images': An array of the top 2 to 4 image URLs (if available in the crawled data).
    """,
    agent=research_agent,
    tools=[firecrawl_tool],
    verbose=True,
)

research_manager = Agent(
    role="Social Media Research Manager",
    goal="""
    Oversee and coordinate the research and content creation workflow for social media posts.

    Responsibilities:
    - Delegate tasks to specialized agents:
      - **Serper Search Agent**: Conducts Google searches and extracts URLs.
      - **Research Agent**: Processes URLs to gather and format content, selecting the top 2 to 4 image URLs(If Available).

    - Ensure efficient task execution and smooth workflow management.
    """,
    backstory="""
    As an AI-powered project manager, you excel in coordinating research, content creation, and image selection tasks.
    Your primary role is to assign tasks effectively and ensure the workflow progresses smoothly.
    """,
    verbose=True,
    memory=True,
    llm=llm_creator,
    allow_delegation=True,
)

research_crew = Crew(
    agents=[serper_search_agent, research_agent],
    tasks=[serper_search_task, research_task],
    verbose=True,
    process=Process.hierarchical,
    manager_agent=research_manager,
)

In [ ]:
result = research_crew.kickoff(inputs={'topic': 'Kerala Tourism', 'platform': 'Instagram'})

In [ ]:
print(result)

In [ ]:
facebook_agent = Agent(
    role="Facebook Content Creator",
    goal="Generate an engaging Facebook post using the provided content and images, with relevant hashtags.",
    backstory="""
    You specialize in crafting highly shareable Facebook content. 
    Use the provided content and images to create a post optimized for engagement, including a set of relevant hashtags.
    """,
    verbose=True,
    memory=True,
    llm=llm_creator,
    allow_delegation=False,
)

instagram_agent = Agent(
    role="Instagram Content Creator",
    goal="Create a visually appealing Instagram post using the provided content and images, with engaging hashtags.",
    backstory="""
    You are a creative Instagram expert, skilled in curating visually appealing posts. 
    Use the provided content and images to create an engaging Instagram post with well-researched hashtags.
    """,
    verbose=True,
    llm=llm_creator,
    allow_delegation=False,
)

twitter_agent = Agent(
    role="Twitter Content Creator",
    goal="Craft a concise and engaging tweet using the provided content and images, with strategic hashtags.",
    backstory="You are an expert in creating highly shareable tweets. Use the provided content and images to craft an engaging tweet with strategic hashtags.",
    verbose=True,
    llm=llm_creator,
    allow_delegation=False,
)

linkedin_agent = Agent(
    role="LinkedIn Content Creator",
    goal="Generate a professional LinkedIn post using the provided content and images, with industry-relevant hashtags.",
    backstory="You are a LinkedIn business content expert. Use the provided content and images to create a professional LinkedIn post with industry-relevant hashtags.",
    verbose=True,
    llm=llm_creator,
    allow_delegation=False,
)

facebook_task = Task(
    description="Create a Facebook post using the provided content and images, including relevant hashtags.",
    expected_output="A high-quality Facebook post with text, a set of relevant hashtags and a list of image url's (is available).",
    agent=facebook_agent,
)

instagram_task = Task(
    description="Generate an Instagram post using the provided content and images, with a compelling caption and hashtags.",
    expected_output="A well-crafted Instagram post with a caption, a set of well-researched hashtags and list of image url's (if available).",
    agent=instagram_agent,
)

twitter_task = Task(
    description="Generate a Twitter post using the provided content and images, including strategic hashtags.",
    expected_output="A high-impact tweet with a message, and strategic hashtags and list of image url's (if available).",
    agent=twitter_agent,
)

linkedin_task = Task(
    description="Generate a LinkedIn post using the provided content and images, with professional hashtags.",
    expected_output="A professional LinkedIn post with content, industry-relevant hashtags and list of image url's (if available).",
    agent=linkedin_agent,
)


post_manager = Agent(
    role="Social Media Project Manager",
    goal="""
    Efficiently manage the creation of a social media post for the specified platform using content and images from the research crew.

    - Assign tasks to the appropriate content creator agent based on the platform:
      - **Facebook Content Creator** for Facebook posts.
      - **Instagram Content Creator** for Instagram posts.
      - **Twitter Content Creator** for Twitter posts.
      - **LinkedIn Content Creator** for LinkedIn posts.

    - Use the JSON input from the research crew, which includes 'content' and 'images', to guide the post creation.
    - Ensure the post is optimized for engagement and aligns with platform-specific best practices.
    """,
    backstory="""
    You are an experienced AI-powered manager, skilled in overseeing content creation for social media. 
    Your role is to efficiently assign tasks to specialized agents and ensure the workflow progresses smoothly.
    """,
    verbose=True,
    memory=True,
    llm=llm_creator,
    allow_delegation=True,
)

post_crew = Crew(
    agents=[facebook_agent, instagram_agent, twitter_agent, linkedin_agent],
    tasks=[facebook_task, instagram_task, twitter_task, linkedin_task],
    process=Process.hierarchical,
    verbose=True,
    manager_agent=post_manager
)

In [ ]:
from pydantic import BaseModel
from crewai.flow import Flow, listen, start
from typing import Dict, List
import json

class PostState(BaseModel):
    content: Dict[str, str] = {}
    images: List[str] = []
    platform: str = ""
    post: str = ""

class PostFlow(Flow[PostState]):

    @start()
    def generate_content(self):
        print("Generating content")
        result = (
            research_crew.kickoff(inputs={'topic': 'Kerala Tourism', 'platform': 'Instagram'})
        )

        print("Post generated", result.raw)
        self.state.content = result.raw

    @listen(generate_content)
    def generate_post(self, content):
        print("Generating post")
           
        result = post_crew.kickoff(content)
        
        print("Post generated", result.raw)
        print(f"Post: {result.raw}")
        self.state.post = result.raw

    @listen(generate_post)
    def save_post(self, post):
        print("Saving Post")
        if post is None:
            print("Post is None, skipping write operation.")
            return  # Exit the function early if post is None
        
        with open("plan.md", "w", encoding="utf-8") as f:
            f.write(post)

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

def kickoff():
    post_flow = PostFlow()
    asyncio.run(post_flow.kickoff_async())  # Use kickoff_async directly

def plot():
    post_flow = PostFlow()
    post_flow.plot()

if __name__ == "__main__":
    kickoff()